# New Computational Approaches to Morphometrics: Combining 3D Complex Shape Representation and Machine Learning for Shape Analysis

Hector A. Orengo 1,2,3,*; Javier Esmoris 4,5; Iban Berganzo-Besga 6,1,2; Felipe Lumbreras 4,5; Paloma Aliende 2; Michael Wallace 7,2; Alexandra Livarda 2

1. Computational Social Sciences and Humanities Department, Barcelona Supercomputing Center - Centro Nacional de Supercomputación (BSC-CNS), Plaça d'Eusebi Güell, 1-3, Les Corts, 08034, Barcelona, Spain

2. Landscape Archaeology Research Group (GIAP), Catalan Institute of Classical Archaeology (ICAC), Plaça Rovellat s/n, 43003, Tarragona, Spain

3. Catalan Institution for Research and Advanced Studies (ICREA), Passeig Lluís Companys 23, 08010, Barcelona, Spain

4. Computer Vision Center (CVC), 08193, Bellaterra (Cerdanyola del Vallès), Spain

5. Department of Computer Science, Universitat Autònoma de Barcelona (UAB), 08193, Bellaterra (Cerdanyola del Vallès), Spain

6. Ramsey Laboratory for Environmental Archaeology (RLEA), University of Toronto Mississauga (UTM), 3359 Mississauga Road, Mississauga, ON, L5L 1C6, Canada

7. Headland Archaeology (UK) Ltd.

\* Correspondence author


---

# MeshCNN

Multi-GPU implementation shown as single-GPU for free Colab usage.

# 1. Initialization

---

In [ ]:
# GitHub Repositories #

!git clone https://github.com/iberganzo/ArchaeoDeep3D

!git clone https://github.com/ranahanocka/MeshCNN

Cloning into 'ArchaeoDeep3D'...
remote: Enumerating objects: 1296, done.
remote: Counting objects: 100% (826/826), done.
remote: Compressing objects: 100% (707/707), done.
remote: Total 1296 (delta 43), reused 785 (delta 34), pack-reused 470 (from 1)
Receiving objects: 100% (1296/1296), 129.82 MiB | 25.87 MiB/s, done.
Resolving deltas: 100% (58/58), done.
Cloning into 'MeshCNN'...
remote: Enumerating objects: 213, done.
remote: Counting objects: 100% (213/213), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 213 (delta 93), reused 167 (delta 88), pack-reused 0 (from 0)
Receiving objects: 100% (213/213), 3.55 MiB | 7.86 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [ ]:
# Code
%cd MeshCNN
!mkdir scripts/seeds/
!cp scripts/shrec/* scripts/seeds/
!rm scripts/seeds/test.sh
!cp ../ArchaeoDeep3D/MeshCNN/Code/test.sh scripts/seeds/
!rm test.py
!cp ../ArchaeoDeep3D/MeshCNN/Code/test.py .
!rm models/layers/mesh_pool.py
!cp ../ArchaeoDeep3D/MeshCNN/Code/mesh_pool.py models/layers/
!cp ../ArchaeoDeep3D/MeshCNN/Code/check_edges.py .

# Dataset
!mkdir datasets/

# Weight
!mkdir checkpoints/
!mkdir checkpoints/testweights/

/content/MeshCNN


In [ ]:
# Imports #

import os
import time
import torch
from options.train_options import TrainOptions
from options.test_options import TestOptions
from data import DataLoader
from util.writer import Writer
from os.path import join
from util.util import seg_accuracy, print_network
from models import networks
import torch.nn as nn
import torch.optim as optim

print("Pytorch version: ", torch.__version__) # 2.5.1+cu121

tensorboard X not installed, visualizing wont be available
Pytorch version:  2.5.1+cu121


In [ ]:
!nvidia-smi

import torch

print("\n")
if torch.cuda.is_available():
    print("Available GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected")

Mon Nov 25 10:44:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 2. Processing

---

In [ ]:
# Rows #

# Code
!cp ../ArchaeoDeep3D/MeshCNN/Code/script_test_rows.py .
!mv script_test_rows.py script_test.py

# Dataset
%cd /content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /
!cp -r Rows /content/MeshCNN/datasets/
%cd /content/MeshCNN/
!mv /content/MeshCNN/datasets/Rows /content/MeshCNN/datasets/testseeds
!mv /content/MeshCNN/datasets/testseeds/2Row/test /content/MeshCNN/datasets/testseeds/2Row/train
!mv /content/MeshCNN/datasets/testseeds/6Row/test /content/MeshCNN/datasets/testseeds/6Row/train

# Weight
!cp ../ArchaeoDeep3D/MeshCNN/Weight/10R_Harmonics_Rows_net.pth checkpoints/testweights/
!mv checkpoints/testweights/10R_Harmonics_Rows_net.pth checkpoints/testweights/1_net.pth

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics 
/content/MeshCNN


In [ ]:
!python3 check_edges.py

max_edges:  9000
min_edges:  9000
max_faces:  4500
ninput_edges:  9000
pool_res:  7200 6300 5400 4500


In [ ]:
!python3 script_test.py

GENERAL ACCURACY
tensorboard X not installed, visualizing wont be available
Running Test
loaded mean / std from cache
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
loading the model from ./checkpoints/testweights/1_net.pth
/content/MeshCNN/test.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

In [ ]:
!rm script_test.py
!rm -r /content/MeshCNN/datasets/*

In [ ]:
# Origin #

# Code
!cp ../ArchaeoDeep3D/MeshCNN/Code/script_test_origin.py .
!mv script_test_origin.py script_test.py

# Dataset
%cd /content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /
!cp -r Origin /content/MeshCNN/datasets/
%cd /content/MeshCNN/
!mv /content/MeshCNN/datasets/Origin /content/MeshCNN/datasets/testseeds
!mv /content/MeshCNN/datasets/testseeds/Dundee/test /content/MeshCNN/datasets/testseeds/Dundee/train
!mv /content/MeshCNN/datasets/testseeds/Orkney/test /content/MeshCNN/datasets/testseeds/Orkney/train

# Weight
!cp ../ArchaeoDeep3D/MeshCNN/Weight/10R_Harmonics_Origin_net.pth checkpoints/testweights/
!mv checkpoints/testweights/10R_Harmonics_Origin_net.pth checkpoints/testweights/1_net.pth

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics 
/content/MeshCNN


In [ ]:
!python3 script_test.py

GENERAL ACCURACY
tensorboard X not installed, visualizing wont be available
Running Test
loaded mean / std from cache
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
loading the model from ./checkpoints/testweights/1_net.pth
/content/MeshCNN/test.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

In [ ]:
!rm script_test.py
!rm -r /content/MeshCNN/datasets/*

In [ ]:
# Bere #

# Code
!cp ../ArchaeoDeep3D/MeshCNN/Code/script_test_bere.py .
!mv script_test_bere.py script_test.py

# Dataset
%cd /content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /
!cp -r Bere /content/MeshCNN/datasets/
%cd /content/MeshCNN/
!mv /content/MeshCNN/datasets/Bere /content/MeshCNN/datasets/testseeds
!mv /content/MeshCNN/datasets/testseeds/Bere/test /content/MeshCNN/datasets/testseeds/Bere/train
!mv /content/MeshCNN/datasets/testseeds/NoBere/test /content/MeshCNN/datasets/testseeds/NoBere/train

# Weight
!cp ../ArchaeoDeep3D/MeshCNN/Weight/10R_Harmonics_Bere_net.pth checkpoints/testweights/
!mv checkpoints/testweights/10R_Harmonics_Bere_net.pth checkpoints/testweights/1_net.pth

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics 
/content/MeshCNN


In [ ]:
!python3 script_test.py

GENERAL ACCURACY
tensorboard X not installed, visualizing wont be available
Running Test
loaded mean / std from cache
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
loading the model from ./checkpoints/testweights/1_net.pth
/content/MeshCNN/test.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

In [ ]:
!rm script_test.py
!rm -r /content/MeshCNN/datasets/*

In [ ]:
# Landraces #

# Code
!cp ../ArchaeoDeep3D/MeshCNN/Code/script_test_landraces.py .
!mv script_test_landraces.py script_test.py

# Dataset
%cd /content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /
!cp -r Landraces /content/MeshCNN/datasets/
%cd /content/MeshCNN/
!mv /content/MeshCNN/datasets/Landraces /content/MeshCNN/datasets/testseeds
!mv /content/MeshCNN/datasets/testseeds/Bere/test /content/MeshCNN/datasets/testseeds/Bere/train
!mv /content/MeshCNN/datasets/testseeds/British/test /content/MeshCNN/datasets/testseeds/British/train
!mv /content/MeshCNN/datasets/testseeds/Scandinavian/test /content/MeshCNN/datasets/testseeds/Scandinavian/train
!mv /content/MeshCNN/datasets/testseeds/Scottish/test /content/MeshCNN/datasets/testseeds/Scottish/train

# Weight
!cp ../ArchaeoDeep3D/MeshCNN/Weight/10R_Harmonics_Landraces_net.pth checkpoints/testweights/
!mv checkpoints/testweights/10R_Harmonics_Landraces_net.pth checkpoints/testweights/1_net.pth

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics 
/content/MeshCNN


In [ ]:
!python3 script_test.py

GENERAL ACCURACY
tensorboard X not installed, visualizing wont be available
Running Test
loaded mean / std from cache
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
loading the model from ./checkpoints/testweights/1_net.pth
/content/MeshCNN/test.py:77: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more det

In [ ]:
!rm script_test.py
!rm -r /content/MeshCNN/datasets/*

In [ ]:
# Visualisation #

!pip install trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.4/704.4 kB 14.5 MB/s eta 0:00:00


In [ ]:
import os
import trimesh
import matplotlib.pyplot as plt

def dataset_sample ():

  obj_files = [f for f in os.listdir("test") if f.endswith('.obj')]
  if obj_files:
      first_obj_file = os.path.join("test", obj_files[0])
  else:
      print("No files")

  return first_obj_file

%cd /content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /
%cd Landraces/Bere/
first_bere = dataset_sample ()
mesh1 = trimesh.load(first_bere)
mesh1.show()

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics 
/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /Landraces/Bere


In [ ]:
%cd ../British/
first_british = dataset_sample ()
mesh2 = trimesh.load(first_british)
mesh2.show()

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /Landraces/British


In [ ]:
%cd ../Scandinavian/
first_scandinavian = dataset_sample ()
mesh3 = trimesh.load(first_scandinavian)
mesh3.show()

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /Landraces/Scandinavian


In [ ]:
%cd ../Scottish/
first_scottish = dataset_sample ()
mesh4 = trimesh.load(first_scottish)
mesh4.show()

/content/ArchaeoDeep3D/MeshCNN/Dataset/10R/Harmonics /Landraces/Scottish


In [ ]:
%cd /content/MeshCNN/

/content/MeshCNN


# 3. References

---

1. Berganzo-Besga, I. 2024. ArchaeoDeep3D: Multi-GPU MeshCNN Deep Learning 3D Classification Algorithm Implementation for Shape Analysis. GitHub repository. Available online: https://github.com/iberganzo/ArchaeoDeep3D (Accessed 25 November 2024)
2. Hanocka R. 2019. MeshCNN: Convolutional Neural Network for 3D meshes in PyTorch. Available online: https://github.com/ranahanocka/MeshCNN (Accessed 25 November 2024)